In [ ]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

pd.set_option('display.max_rows',70)
display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def get_file_name(name):
    return name.replace(':', '-')

In [ ]:
inputFile = 'collected-data.csv'

In [ ]:
df = pd.read_csv(inputFile)
df

,timestamp,sensor_type,sensor_name,value,accuracy
0,1620723880333,-1,INDOOR,1.0,NaN
1,1620723880364,-1,MONITORING,1.0,NaN
2,1620723873432,5,TMD3725_Light Ambient Light Sensor Non-wakeup,15.0,3.0
3,1620723873265,8,TMD3725_Proximity Proximity Sensor Wakeup,5.0,3.0
4,1620723882262,-1,GPS_SATELLITES,17.0,NaN
...,...,...,...,...,...
2403,1620733476294,5,TMD3725_Light Ambient Light Sensor Non-wakeup,8.0,3.0
2404,1620733477129,8,TMD3725_Proximity Proximity Sensor Wakeup,5.0,3.0
2405,1620733477434,5,TMD3725_Light Ambient Light Sensor Non-wakeup,2.0,3.0
2406,1620733480731,5,TMD3725_Light Ambient Light Sensor Non-wakeup,2.0,3.0


In [ ]:
sensor_type_dict = {
    'MONITORING':-1,
    'INDOOR':-2,
    'GPS_SATELLITES':-3,
    'GPS_FIX_SATELLITES':-4,
    'GPS_FIX':-5,
    'DETECTED_ACTIVITY':-6,
    'WIFI_ACCESS_POINTS':-7,
    'BLUETOOTH_DEVICES':-8
}

for index,row in df.iterrows():
    if  df.loc[index,'sensor_type'] == -1:
        df.loc[index,'sensor_type'] = sensor_type_dict[ row['sensor_name']]

In [ ]:
df_wide = df.pivot_table(index=['timestamp'], columns='sensor_name', values='value')


last_seen_values = {}
sensor_types = df['sensor_name'].unique()
for sensor_type in sensor_types:
    last_seen_values[sensor_type] = float('nan')

for index,row in df_wide.iterrows():
    if row['MONITORING'] == 0:
        for sensor_type in sensor_types:
            last_seen_values[sensor_type] = float('nan')
    else:        
        for sensor_type in sensor_types:
            if math.isnan(row[sensor_type]):
                row[sensor_type] = last_seen_values[sensor_type]
            last_seen_values[sensor_type] = row[sensor_type] 

for index,row in df_wide.iterrows():
    containsNaN = 0.0
    for sensor_type in sensor_types:
        if math.isnan(row[sensor_type]):
            containsNaN = 1.0
            break
    df_wide.loc[index,'containsNaN'] = containsNaN

df_wide = df_wide[(df_wide['MONITORING'] == 1.0) & (df_wide['containsNaN'] == 0.0)] 
del df_wide['containsNaN']

proximity_name = ""
light_name = ""
for sensor_type in sensor_types:
    if 'proximity' in sensor_type.lower():
        proximity_name = sensor_type
        break
    if 'light' in sensor_type.lower():
        light_name = sensor_type
        
df_wide.loc[(df_wide[proximity_name] != 0.0),proximity_name] = 1

Detected_Activity_List = {
  0.0 : "IN_VEHICLE",
  1.0 : "ON_BICYCLE",
  2.0 : "ON_FOOT",
  3.0 : "STILL",
  4.0 : "UNKOWN",
  5.0 : "TILTING",
  7.0 : "WALKING",
  8.0 : "RUNNING"
}

for key in Detected_Activity_List.keys(): 
  df_wide[Detected_Activity_List[key]] = df_wide['DETECTED_ACTIVITY'] == key

df_wide.sort_values(['timestamp'], inplace=True)
target = df_wide.loc[:,'INDOOR']
del df_wide['INDOOR']
del df_wide['GPS_FIX']
del df_wide['MONITORING']
del df_wide['DETECTED_ACTIVITY']

cols = [sensor_type for sensor_type in df_wide.columns.to_list() if sensor_type != proximity_name]
cols.append(proximity_name)
df_wide = df_wide[cols].replace(True,1.0).replace(False,0.0)
df_wide.to_csv('preprocessed_data.csv')
target.to_numpy()
df_wide

sensor_name,BLUETOOTH_DEVICES,GPS_FIX_SATELLITES,GPS_SATELLITES,TMD3725_Light Ambient Light Sensor Non-wakeup,WIFI_ACCESS_POINTS,IN_VEHICLE,ON_BICYCLE,ON_FOOT,STILL,UNKOWN,TILTING,WALKING,RUNNING,TMD3725_Proximity Proximity Sensor Wakeup
timestamp,,,,,,,,,,,,,,
1620724298381,1.0,0.0,35.0,7.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1620724299464,1.0,0.0,35.0,7.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1620724300115,1.0,0.0,35.0,7.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1620724300588,1.0,0.0,35.0,7.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1620724300600,2.0,0.0,35.0,7.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1620733475300,0.0,0.0,39.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1620733476294,0.0,0.0,39.0,8.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1620733477129,0.0,0.0,39.0,8.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_wide.to_numpy(), target, test_size=0.33) 
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)


(1551, 14) (765, 14) (1551,) (765,)


In [ ]:
D = X_train.shape[1]
X_train

array([[ 3.,  0., 17., ...,  0.,  0.,  1.],
       [ 2.,  0., 17., ...,  0.,  0.,  1.],
       [ 0.,  0., 37., ...,  0.,  0.,  1.],
       ...,
       [ 1.,  0., 36., ...,  0.,  0.,  1.],
       [ 0.,  0., 36., ...,  0.,  0.,  1.],
       [ 1.,  0., 36., ...,  0.,  0.,  1.]])

In [122]:
# Now all the fun Tensorflow stuff
# Build the model
import load_data from preprocessing

X_train,y_train X_test, y_test = load_data() 

i = tf.keras.layers.Input(shape=(D,),name='Input')
n_i = tf.keras.layers.LayerNormalization(axis=-1,name='Normalization')(i[:,:5])
x = tf.keras.layers.Concatenate(name='Concatenate')([n_i,i[:,5:]])
x = tf.keras.layers.Dense(1, activation='sigmoid', name='Classifier')(x)
model = tf.keras.models.Model(i, x)


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

r = model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=200)

print("Train score:", model.evaluate(X_train, y_train))
print("Test score:", model.evaluate(X_test, y_test))


Epoch 1/200
49/49 [==============================] - 1s 6ms/step - loss: 0.9412 - accuracy: 0.4550 - val_loss: 0.8474 - val_accuracy: 0.4497
Epoch 2/200
49/49 [==============================] - 0s 2ms/step - loss: 0.8508 - accuracy: 0.4538 - val_loss: 0.7358 - val_accuracy: 0.5203
Epoch 3/200
49/49 [==============================] - 0s 3ms/step - loss: 0.7281 - accuracy: 0.5319 - val_loss: 0.6571 - val_accuracy: 0.5882
Epoch 4/200
49/49 [==============================] - 0s 3ms/step - loss: 0.6388 - accuracy: 0.5851 - val_loss: 0.6015 - val_accuracy: 0.6314
Epoch 5/200
49/49 [==============================] - 0s 9ms/step - loss: 0.5945 - accuracy: 0.6101 - val_loss: 0.5632 - val_accuracy: 0.6562
Epoch 6/200
49/49 [==============================] - 0s 3ms/step - loss: 0.5650 - accuracy: 0.6542 - val_loss: 0.5371 - val_accuracy: 0.7503
Epoch 7/200
49/49 [==============================] - 0s 2ms/step - loss: 0.5496 - accuracy: 0.7504 - val_loss: 0.5175 - val_accuracy: 0.8301
Epoch 8/200
4

In [123]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [124]:
# Plot the accuracy too
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [125]:
model.summary()


Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 14)]         0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_15 (Sl (None, 5)            0           Input[0][0]                      
__________________________________________________________________________________________________
Normalization (LayerNormalizati (None, 5)            10          tf.__operators__.getitem_15[0][0]
__________________________________________________________________________________________________
tf.__operators__.getitem_16 (Sl (None, 9)            0           Input[0][0]                      
___________________________________________________________________________________________

In [126]:
for z in zip(df_wide.columns.values,model.layers[5].get_weights()[0]):
  print(z)


('BLUETOOTH_DEVICES', array([0.78467166], dtype=float32))
('GPS_FIX_SATELLITES', array([-2.941558], dtype=float32))
('GPS_SATELLITES', array([0.3114541], dtype=float32))
('TMD3725_Light Ambient Light Sensor Non-wakeup', array([-0.46210802], dtype=float32))
('WIFI_ACCESS_POINTS', array([2.3207603], dtype=float32))
('IN_VEHICLE', array([-1.6584494], dtype=float32))
('ON_BICYCLE', array([-0.40118644], dtype=float32))
('ON_FOOT', array([-5.6376424], dtype=float32))
('STILL', array([1.7916169], dtype=float32))
('UNKOWN', array([1.2244202], dtype=float32))
('TILTING', array([0.3653815], dtype=float32))
('WALKING', array([-0.27648577], dtype=float32))
('RUNNING', array([0.12622678], dtype=float32))
('TMD3725_Proximity Proximity Sensor Wakeup', array([0.3069393], dtype=float32))


In [129]:
tf.keras.utils.plot_model(model,show_shapes=True,show_dtype=True, show_layer_names=True, expand_nested=True)